# MiniProyecto 4: Optimización de parámetros


En este proyecto vamos a seguir trabajando (por última vez) con el dataset de propiedades en venta publicadas en el portal [Properati](www.properati.com.ar). El objetivo en este caso es optimizar los parámetros de los algoritmos que usamos en el proyecto pasado.

El dataset es el mismo del proyecto 3. Recordemos que las columnas que se agregan son:

* `barrios_match`: si coincide el barrio publicado con el geográfico vale 1, si no 0.

* `PH`, `apartment`, `house`: variables binarias que indican el tipo de propiedad.

* dummies de barrios: variables binarias con 1 o 0 según el barrio.

La métrica que vamos a usar para medir es RMSE (raíz del error cuadréatico medio), cuya fórmula es:

$$RMSE = \sqrt{\frac{\sum_{t=1}^n (\hat y_t - y_t)^2}{n}}$$

## Pandas - Levantamos el dataset

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.3f' % x)
path_dataset = 'datos_properati_limpios_modelo.csv'
df = pd.read_csv(path_dataset)



**Separá** el dataset en entrenamiento (80%) y test (20%) utilizando como target la columna `price_aprox_usd`

In [2]:
X, y = df.drop(df.columns[2], axis=1), df.iloc[:,2]

from sklearn.model_selection import train_test_split
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.20, random_state= 42)
print(X_entrenamiento.shape[0], y_prueba.shape[0])


5100 1276


In [3]:
import numpy as np
np.random.seed(123)
from sklearn.model_selection import train_test_split
X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print(X_train.shape[0], y_test.shape[0])


5100 1276


## Scikit-learn - Entrenamiento

Para repasar los parámetros de árboles de decisión en Scikit-learn: 

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

En primer lugar veamos como hacer cross validation. Para eso necesitamos definir la cantidad de folds, en este caso vamos a usar 5.

GridSearchCV nos permite testear a través de un espacio de búsqueda de parámetros la mejor combinación posible dado un estimador.

Por ejemplo, en este caso probamos la profundidad máxima y la máxima cantidad de features para hacer los split. Ambos entre 1 y 5.
Recordemos que para hacer la optimización scikit-learn usa la métrica `neg_mean_squared_error` en lugar de `mean_squared_error`.

**Creá** una variable `param_grid` con valores del 1 al 5 para los atributos `max_depth` y `max_features`. 

In [4]:
param_grid = [{'max_depth' : [1,2,3,4,5], 'max_features' : [1,2,3,4,5]}, ]

**Importá** `GridSearchCV` y `DecisionTreeRegressor`.

**Creá** una variable `grid_search` y asignale un `GridSearchCV` que recorra el `param_grid` que creaste con el algoritmos `DecisionTreeRegressor` y el un scoring de `neg_mean_squared_error`

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
grid_search = GridSearchCV (tree_reg, param_grid, cv = 5 , scoring = 'neg_mean_squared_error', return_train_score = True )

In [6]:
grid_search.fit(X_entrenamiento,y_entrenamiento)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'max_depth': [1, 2, 3, 4, 5], 'max_features': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

Revisemos los resultados. Recordemos que no están expresados en RMSE.

In [7]:
grid_search.scorer_(grid_search,X_entrenamiento, y_entrenamiento)


-947511458.6600356

**Mostrá** los `grid_scores` obtenidos durante el `grid_search`

In [8]:
Parametros = pd.Series(grid_search.cv_results_['params'])
Score_Train = pd.Series(grid_search.cv_results_['mean_train_score'])
Score_Test = pd.Series(grid_search.cv_results_['mean_test_score'])
grid_score = pd.concat([Parametros.rename('Parametros'),Score_Train.rename('neg_mean_score_train') ,Score_Test.rename('neg_mean_score_test')] ,  axis=1)
grid_score

,Parametros,neg_mean_score_train,neg_mean_score_test
0,"{'max_depth': 1, 'max_features': 1}",-955588856.766,-967815477.820
1,"{'max_depth': 1, 'max_features': 2}",-957468890.415,-967910484.499
2,"{'max_depth': 1, 'max_features': 3}",-972983861.276,-973432428.277
3,"{'max_depth': 1, 'max_features': 4}",-923628745.537,-926098951.655
4,"{'max_depth': 1, 'max_features': 5}",-974037207.511,-978231079.818
5,"{'max_depth': 2, 'max_features': 1}",-927697741.071,-939192559.533
6,"{'max_depth': 2, 'max_features': 2}",-922782381.801,-915148162.547
7,"{'max_depth': 2, 'max_features': 3}",-918594195.319,-913017431.133
8,"{'max_depth': 2, 'max_features': 4}",-917251310.275,-914864706.669
9,"{'max_depth': 2, 'max_features': 5}",-926891536.633,-926966397.138


De esta manera, el valor con mejor resultado (dado el espacio de búsqueda definido) es `max_depth` 4 y `max_features` 4
.

**Mostramos** el mejor score y los mejores parámetros encontrados por `grid_search`

In [9]:
print ( 'El mejor neg_mean_score es :  ',  grid_search.best_score_ )
print ('Los mejores parametros encontrados han sido :' , grid_search.best_params_)

El mejor neg_mean_score es :   -713698124.9712428
Los mejores parametros encontrados han sido : {'max_depth': 4, 'max_features': 4}


Convertimos a RMSE.

In [10]:
score = grid_search.best_score_ 

def nmsq2rmse(score):
    return np.round(np.sqrt(-score), 2)


nmsq2rmse (score)

26715.13

__Encontrar el mejor modelo para el espacio de búsqueda dado__

* `"min_samples_split": [2, 10, 20]`
* `"max_depth": [None, 2, 5, 10, 15]`
* `"min_samples_leaf": [1, 5, 10, 15]`
* `"max_leaf_nodes": [None, 5, 10, 20]`

Recordemos que `GridSearchCV` tiene como parámetro default `refit=True`. Esto significa que luego de hacer la corrida se ajusta el mejor modelo al conjunto de datos de entrada. De esta manera, se puede predecir directamente usando `best_estimator_`.

In [11]:
params_grid = [ { "min_samples_split": [2, 10, 20],"max_depth": [None, 2, 5, 10, 15],
                 "min_samples_leaf": [1, 5, 10, 15], "max_leaf_nodes": [None, 5, 10, 20]}]
grid_search_2 = GridSearchCV (tree_reg, params_grid, cv = 5 , scoring = 'neg_mean_squared_error', return_train_score = True )
grid_search_2.fit(X_entrenamiento, y_entrenamiento)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'min_samples_split': [2, 10, 20], 'max_depth': [None, 2, 5, 10, 15], 'min_samples_leaf': [1, 5, 10, 15], 'max_leaf_nodes': [None, 5, 10, 20]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [61]:
print ( 'El mejor neg_mean_score es :  ',  grid_search_2.best_score_ )
print ('Los mejores parametros encontrados han sido :' , grid_search_2.best_params_)

El mejor neg_mean_score es :   -459407773.74392384
Los mejores parametros encontrados han sido : {'max_depth': 10, 'max_leaf_nodes': None, 'min_samples_leaf': 15, 'min_samples_split': 2}


In [62]:
optimised_decision_tree = grid_search_2.best_estimator_
optimised_decision_tree


DecisionTreeRegressor(criterion='mse', max_depth=10, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=15,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

__Evaluemos en testing el desempeño de este modelo.__

Como venimos trabajando, el resultado en testing será la medición que usaremos como benchmark para comparar este modelos con otros en el futuro, puesto que no estuvo en contacto con el dataset de test para la calibración.

In [63]:
from sklearn.metrics import mean_squared_error
y_opt_pred = optimised_decision_tree.predict(X_prueba)
rmse = np.sqrt(mean_squared_error(y_prueba, y_opt_pred))
np.round(rmse)

21301.0

Vemos los primeros 10 resultados de la predicción del valor de propiedades.

In [64]:
val_real = pd.Series(y_test.values)
val_pred = pd.Series(y_opt_pred)

In [65]:
predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)')] ,  axis=1)

In [66]:
predicciones.head(10)

,Valor real,Valor Pred,Dif(+/-)
0,80000.000,103438.660,23438.660
1,128000.000,135705.882,7705.882
2,150000.000,156075.759,6075.759
3,85000.000,102400.991,17400.991
4,135000.000,135571.622,571.622
5,135000.000,109560.000,25440.000
6,68000.000,75181.250,7181.250
7,110000.000,140444.444,30444.444
8,134000.000,158431.250,24431.250
9,110000.000,76701.202,33298.798
